In [191]:
import pandas as pd
import numpy as np
import pickle as p

In [192]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=100, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [193]:
# item_subset

### `/train`

In [194]:
import time

In [195]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 3}

Thread(target=requests.post, args=(url, ), kwargs={'data': payload, 'files': files}).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [201]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'


payload = {'project_id': 3}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV Done: 0


### `/get_cv_results`

In [196]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'


payload = {'project_id': 3}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)
pd.DataFrame().from_dict((json.loads(r.json())))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [188]:
r.json()

'{"item_id":{"0":3821.0,"1":7157.0,"2":4725.0,"3":7965.0,"4":5501.0,"5":4270.0,"6":3816.0,"7":204.0,"8":3768.0,"9":16017.0,"10":3731.0,"11":2581.0,"12":2531.0,"13":7114.0,"14":5469.0,"15":3743.0,"16":2867.0,"17":16018.0,"18":2701.0,"19":201.0,"20":4978.0,"21":7128.0,"22":3732.0,"23":7109.0,"24":3772.0,"25":3726.0,"26":7575.0,"27":7434.0,"28":2521.0,"29":3766.0,"30":202.0,"31":3733.0,"32":2731.0,"33":2589.0,"34":3727.0,"35":1195.0,"36":3728.0,"37":2316.0,"38":7008.0,"39":350.0,"40":1221.0,"41":5581.0,"42":16019.0,"43":4272.0,"44":2256.0,"45":1230.0,"46":3773.0,"47":2257.0,"48":3045.0,"49":102.0},"avg_sales":{"0":19.5379333496,"1":12.5704746246,"2":15.5809736252,"3":38.1437644958,"4":468.8437194824,"5":45.5170135498,"6":47.339138031,"7":27.3373699188,"8":257.9417114258,"9":17.3660087585,"10":19.2884159088,"11":256.5289916992,"12":321.2454833984,"13":23.6277160645,"14":27.6312141418,"15":134.0812072754,"16":948.2962036133,"17":84.9915161133,"18":487.2629089355,"19":56.8451728821,"20":107.

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [203]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
# a_input


# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 3}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

Input Prices Quantities:


,Estimated,Actual
Qty_1004,2732.0,3121.0
Qty_102,184.0,211.0
Qty_11,173.0,153.0
Qty_1152,52.0,42.0
Qty_1195,81.0,76.0
...,...,...
Qty_7434,63.0,61.0
Qty_7575,83.0,70.0
Qty_7965,24.0,18.0
Qty_7966,56.0,53.0
